# 유튜브 랭킹 데이터 수집


In [99]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [100]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [101]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [102]:
url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube'
driver.get(url)
# time.sleep(2)

In [103]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

100

In [104]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [105]:
# soup.select('form > table > tbody > tr')
channel_list = soup.select('.aos-init')
len(channel_list)

100

In [106]:
channel= channel_list[0]
category= channel.select_one('p.category').get_text().strip()
category

'[음악/댄스/가수]'

In [107]:
name = channel.select_one('.subject a').text.strip()
name

'BLACKPINK'

In [108]:
subscriber = channel.select_one('.subscriber_cnt').text
view = channel.select_one('.view_cnt').text
video = channel.select_one('.video_cnt').text[:-1]
subscriber, view, video

('6400만', '190억0381만', '371')

* 한 페이지에 있는 내용 가져오기

In [109]:
channels= []
for channel in channel_list:
  category= channel.select_one('p.category').get_text().strip(' \n[]')
  name = channel.select_one('.subject a').text.strip()
  subscriber = channel.select_one('.subscriber_cnt').text
  view = channel.select_one('.view_cnt').text
  video = channel.select_one('.video_cnt').text
  channels.append([category, name, subscriber, view, video])


In [110]:
df= pd.DataFrame(channels, columns=['카테고리', '채널명','구독자수', '조회수', '비디오수'])
df.head()

,카테고리,채널명,구독자수,조회수,비디오수
0,음악/댄스/가수,BLACKPINK,6400만,190억0381만,371개
1,음악/댄스/가수,HYBE LABELS,6030만,187억1305만,654개
2,음악/댄스/가수,BANGTANTV,5640만,121억9496만,"1,579개"
3,음악/댄스/가수,SMTOWN,2850만,218억7092만,"3,729개"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그],2650만,110억5288만,223개


* 만과 억을 숫자로 바꿔주는 함수

In [95]:
def convert_unit(s):
    #s = ''.join(s.split('억'))
    s = s.replace('억', '').replace('개','').replace(',','')
    s = s.replace('만', '0000')
    return f'{int(s):,d}'

In [96]:
convert_unit('110억5288만')

'11,052,880,000'

* 두번째 페이지 - xPath로 찾아 click() 하여 이동하기

In [111]:
driver.find_element_by_xpath('//*[@id="list-skin"]/nav/span/a[1]').click()

In [112]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [113]:
channel_list= soup.select('.aos-init')
len(channel_list)

100

In [116]:
result= []
for channel in channel_list:
  category= channel.select_one('p.category').get_text().strip(' \n[]')
  name = channel.select_one('.subject a').text.strip()
  subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
  view = convert_unit(channel.select_one('.view_cnt').text)
  video = convert_unit(channel.select_one('.video_cnt').text)
  result.append([category, name, subscriber, view, video])


In [118]:
df= pd.DataFrame(result, columns=['카테고리', '채널명','구독자수', '조회수', '비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
95,음식/요리/레시피,푸드킹덤 Food Kingdom,"2,090,000","744,530,000",797
96,음악/댄스/가수,4Minute 포미닛(Official YouTube Channel),"2,090,000","1,236,000,000",82
97,미분류,김종국 GYM JONG KOOK,"2,080,000","54,080,000",10
98,음식/요리/레시피,ZOEY ASMR 조이,"2,070,000","475,380,000",606
99,영화/만화/애니,레이디버그TV,"2,070,000","1,087,450,000",325


## page 1에서 1까지 크롤링하기


In [85]:
results = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    channel_list = soup.select('.aos-init')

    for channel in channel_list:
        category = channel.select_one('p.category').get_text().strip(' \n[]')
        name = channel.select_one('.subject a').text.strip()
        subscriber = convert_unit(channel.select_one('.subscriber_cnt').text)
        view = convert_unit(channel.select_one('.view_cnt').text)
        video = convert_unit(channel.select_one('.video_cnt').text)
        results.append([category,name,subscriber,view,video])

In [86]:
df= pd.DataFrame(results, columns=['카테고리', '채널명','구독자수', '조회수', '비디오수'])
df.tail()

,카테고리,채널명,구독자수,조회수,비디오수
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble],"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"
999,게임,임선비,"510,000","234,290,000","1,134"


In [87]:
df.to_csv('유튜브_순위.csv', index=None)

In [119]:
driver.close()